In [1]:
import os
import numpy as np
import copy
import time

from curbside_models import *

## Initialization

In [2]:
Q = 2 # Maximum number of spaces at each candidate location
B = 50000 # ($) annual budget
H = 100 # average daily pickups/dropoffs for 1 space

n_rows = 16
n_cols = 37

candidate_list = [(14, 3), (11, 5), (11, 7), (9, 9), (4, 11), (6, 13), (3, 14), 
                  (6, 16), (4, 17), (2, 21), (4, 22), (5, 27), (0, 28), (5, 32)]
costs_list = [(2000, 5000) for i in range(len(candidate_list))]
srkernel_list = [[1, 0.6, 0.2, 0] for i in range(len(candidate_list))]

np.random.seed(100)
demand_matrix = np.random.randint(50, size=(n_rows, n_cols))

# remove demands far from all candidate locations
for row in range(n_rows):
    for col in range(n_cols):
        _keep = False
        for k in range(len(candidate_list)):
            i = candidate_list[k][0]
            j = candidate_list[k][1]
            if abs(row-i) + abs(col-j) < len(srkernel_list[k]):
                _keep = True
                break
        if not _keep:
            demand_matrix[row][col] = 0

g = Grid(n_rows, n_cols, candidate_list, costs_list, srkernel_list, demand_matrix)

## Exaustive Search

In [6]:
start = time.time()

n_cand = len(candidate_list)
min_obj = 999999999
min_cand = []

def obj_value(sol):
    g.clear_supply()
    for i in range(n_cand):
        if sol[i] == 1:
            g.update_supply(candidate_list[i][0], candidate_list[i][1], H)
    return g.unmet_demand()

def generate_solution(sol):
    global min_obj
    global min_cand
    if len(sol) == n_cand:
        _cost = 0
        for i in range(n_cand):
            if sol[i] > 0:
                _cost += g.cells[candidate_list[i][0]][candidate_list[i][1]].fixed_cost
                _cost += g.cells[candidate_list[i][0]][candidate_list[i][1]].operational_cost * sol[i]
        if _cost <= B and obj_value(sol) < min_obj:
            min_obj = obj_value(sol)
            min_cand = copy.copy(sol)
    else:
        sol1 = copy.copy(sol)
        sol1.append(0)
        generate_solution(sol1)
        sol2 = copy.copy(sol)
        sol2.append(1)
        generate_solution(sol2)
        
generate_solution([])
print(min_obj)
print(min_cand)

end = time.time()
print(end - start)

5187.0
[0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1]
9.921203136444092


## Heuristics/ Greedy algorithm

In [8]:
start = time.time()

g.clear_supply()
budget_left = B
min_obj2 = 0
min_cand2 = [0 for i in range(n_cand)]
Q = 1

while budget_left > 0:
    max_contribution = 0
    _index = -1
    for i in range(n_cand):
        if min_cand2[_index] < Q and g.cells[candidate_list[i][0]][candidate_list[i][1]].fixed_cost + \
           g.cells[candidate_list[i][0]][candidate_list[i][1]].operational_cost < budget_left:
            _tmp = g.contribution_of_one_cadidate(candidate_list[i][0], candidate_list[i][1], H)
            if _tmp > max_contribution:
                max_contribution = _tmp
                _index = i
    budget_left -= g.cells[candidate_list[i][0]][candidate_list[i][1]].fixed_cost + g.cells[candidate_list[i][0]][candidate_list[i][1]].operational_cost
    if _index >= 0:
        min_cand2[_index] += 1
        g.update_supply(candidate_list[_index][0], candidate_list[_index][1], H) 
    else:
        break

print(g.unmet_demand())
print(min_cand2)

end = time.time()
print(end - start)

5187.0
[0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0]
0.009006738662719727
